In [268]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
from dstapi import DstApi


C:\Users\echic\AppData\Local\Temp\ipykernel_1516\4002683430.py:4: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


Create objects to interact with API of Denmark Statistics and display a summary table of the datasets informations for the GDP dataset:

In [269]:
gdp_dst = DstApi('NRHP')

t_gdp = gdp_dst.tablesummary(language='en')
display(t_gdp)

Table NRHP: 1-2.1.1 Production, GDP and generation of income by region, transaction, price unit and time
Last update: 2022-10-27T08:00:00


,variable name,# values,First value,First value label,Last value,Last value label,Time variable
0,OMRÅDE,18,000,All Denmark,999,Outside regions,False
1,TRANSAKT,9,P1K,P.1 Output,B2A3GD,B.2g+B.3g Gross operating surplus and mixed in...,False
2,PRISENHED,4,V_T,"Current prices, (mill. DKK.)",LRG_C,"Pr. capita, 2010-prices, chained values, (1000...",False
3,Tid,29,1993,1993,2021,2021,True


Look up values (rows) that each variable (columns) can take: 

In [270]:
for variable in t_gdp['variable name']:
    print(variable+':')
    display(gdp_dst.variable_levels(variable, language='en'))

OMRÅDE:


,id,text
0,000,All Denmark
1,084,Region Hovedstaden
2,01,Province Byen København
3,02,Province Københavns omegn
4,03,Province Nordsjælland
5,04,Province Bornholm
6,085,Region Sjælland
7,05,Province Østsjælland
8,06,Province Vest- og Sydsjælland
9,083,Region Syddanmark


TRANSAKT:


,id,text
0,P1K,P.1 Output
1,P2D,P.2 Intermediate consumption
2,B1GD,B.1g Gross value added
3,D21X31D,D.21-D.31 Taxes less subsidies on products
4,B1GQD,B.1*g Gross domestic product
5,D29X39D,D.29-D.39 Other taxes less subsidies on produc...
6,B1GFD,B.1GF Gross domestic product at factor cost
7,D1D,D.1 Compensation of employees
8,B2A3GD,B.2g+B.3g Gross operating surplus and mixed in...


PRISENHED:


,id,text
0,V_T,"Current prices, (mill. DKK.)"
1,V_C,"Pr. capita. Current prices, (1000 DKK.)"
2,LRG_T,"2010-prices, chained values, (mill. DKK.)"
3,LRG_C,"Pr. capita, 2010-prices, chained values, (1000..."


Tid:


,id,text
0,1993,1993
1,1994,1994
2,1995,1995
3,1996,1996
4,1997,1997
5,1998,1998
6,1999,1999
7,2000,2000
8,2001,2001
9,2002,2002


Look up the format of the dictionary of dataset parameters:

In [271]:
par_gdp = gdp_dst._define_base_params(language='en')

display(par_gdp)

{'table': 'nrhp',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'OMRÅDE', 'values': ['*']},
  {'code': 'TRANSAKT', 'values': ['*']},
  {'code': 'PRISENHED', 'values': ['*']},
  {'code': 'Tid', 'values': ['*']}]}

Define parameters dictionary to select only specified values (rows) of dataset:

In [272]:
par_gdp = {'table': 'nrhp',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'OMRÅDE', 'values': ['000']},
  {'code': 'TRANSAKT', 'values': ['B1GQD']},
  {'code': 'PRISENHED', 'values': ['V_C']},
  {'code': 'Tid', 'values': ['>1993<=2021']}]}

# Just took some random parameters: I'll fix it later
# !! Does real price in gdp dataset have same base year as in consumption dataset?

Download dataset using only specified parameters:

In [273]:
gdp = gdp_dst.get_data(params=par_gdp)

display(gdp.head(5))

,OMRÅDE,TRANSAKT,PRISENHED,TID,INDHOLD
0,All Denmark,B.1*g Gross domestic product,"Pr. capita. Current prices, (1000 DKK.)",1994,191
1,All Denmark,B.1*g Gross domestic product,"Pr. capita. Current prices, (1000 DKK.)",1995,198
2,All Denmark,B.1*g Gross domestic product,"Pr. capita. Current prices, (1000 DKK.)",1996,207
3,All Denmark,B.1*g Gross domestic product,"Pr. capita. Current prices, (1000 DKK.)",1997,217
4,All Denmark,B.1*g Gross domestic product,"Pr. capita. Current prices, (1000 DKK.)",1998,224


Renaming columns:

In [274]:
gdp.rename(columns = {'OMRÅDE':'Area', 
                      'PRISENHED':'Price unit', 
                      'TID':'Variables', #helpfull later
                      'INDHOLD':'GDP'}, inplace=True)
gdp.head(5)

,Area,TRANSAKT,Price unit,Variables,GDP
0,All Denmark,B.1*g Gross domestic product,"Pr. capita. Current prices, (1000 DKK.)",1994,191
1,All Denmark,B.1*g Gross domestic product,"Pr. capita. Current prices, (1000 DKK.)",1995,198
2,All Denmark,B.1*g Gross domestic product,"Pr. capita. Current prices, (1000 DKK.)",1996,207
3,All Denmark,B.1*g Gross domestic product,"Pr. capita. Current prices, (1000 DKK.)",1997,217
4,All Denmark,B.1*g Gross domestic product,"Pr. capita. Current prices, (1000 DKK.)",1998,224


Dropping unimportant variables:

In [275]:
gdp.drop(['TRANSAKT', 'Area', 'Price unit'], axis='columns', inplace=True)
gdp.head(5)

,Variables,GDP
0,1994,191
1,1995,198
2,1996,207
3,1997,217
4,1998,224


Change year to not-only-number and set index:

In [276]:
import string 
for value in gdp['Variables'].values:
    gdp.loc[gdp['Variables'].values == value,['Variables']] = 'year'+str(value)
gdp = gdp.set_index('Variables')
gdp.head(5)

,GDP
Variables,
year1994,191
year1995,198
year1996,207
year1997,217
year1998,224


Transpose:

In [277]:
gdp = gdp.T
gdp.head(5)

Variables,year1994,year1995,year1996,year1997,year1998,year1999,year2000,year2001,year2002,year2003,...,year2012,year2013,year2014,year2015,year2016,year2017,year2018,year2019,year2020,year2021
GDP,191,198,207,217,224,233,249,256,262,267,...,339,344,351,358,368,380,389,397,399,428


Import dataset for "consumption choices" previously downloaded from DST, selecting only the necessary parameters. We also skip empty rows:

In [278]:
filename = 'FU07_cp.xlsx'
cop = pd.read_excel(filename, skiprows=2, skipfooter=2)
display(cop.head(50))

,Unnamed: 0,Unnamed: 1,Unnamed: 2,1994,1995,1996,1997,1998,1999,2000,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Current prices,Average Households,01.1 Food,22325,23280,23488,24021,24904,24905,25605,...,33133,32802,33381,33393,33268,33526,35068,34380,35485,34864
1,NaN,NaN,"04.5 Electricity, gas and other fuels",11551,11786,13261,14056,14669,14579,15426,...,23866,24282,24322,22306,20812,23919,23940,23580,19925,25635
2,NaN,NaN,05.2 Household textiles,920,916,852,870,851,907,915,...,1278,1215,1275,1496,1307,994,1064,1116,1408,1723
3,NaN,NaN,07.1 Purchase of vehicles,12170,12029,12453,13525,13535,14268,10072,...,14669,15307,13928,17255,15671,15465,17742,17754,21668,20965
4,NaN,NaN,07.3 Transport services,3663,3701,3445,3720,3425,3669,3869,...,6188,6096,5820,4841,4533,5133,6010,5918,3324,3026
5,NaN,NaN,12.1 Personal care,4092,4274,4531,4697,4874,5072,5169,...,6163,6236,5916,6283,5945,6522,6733,6155,5970,5761
6,NaN,NaN,12.5 Insurance,5875,6206,6528,7519,8344,8809,8399,...,15398,14502,14820,18237,15276,14183,15225,15210,15887,19317


We drop NaN columns:

In [279]:
drop_these = ['Unnamed: ' + str(num) for num in range(2)] # use list comprehension to create list of columns
cop.drop(drop_these, axis=1, inplace=True) # axis = 1 -> columns, inplace=True -> changed, no copy made
cop.head(10)

,Unnamed: 2,1994,1995,1996,1997,1998,1999,2000,2001,2002,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,01.1 Food,22325,23280,23488,24021,24904,24905,25605,25803,25972,...,33133,32802,33381,33393,33268,33526,35068,34380,35485,34864
1,"04.5 Electricity, gas and other fuels",11551,11786,13261,14056,14669,14579,15426,17058,17774,...,23866,24282,24322,22306,20812,23919,23940,23580,19925,25635
2,05.2 Household textiles,920,916,852,870,851,907,915,1071,1272,...,1278,1215,1275,1496,1307,994,1064,1116,1408,1723
3,07.1 Purchase of vehicles,12170,12029,12453,13525,13535,14268,10072,9742,10985,...,14669,15307,13928,17255,15671,15465,17742,17754,21668,20965
4,07.3 Transport services,3663,3701,3445,3720,3425,3669,3869,3984,4154,...,6188,6096,5820,4841,4533,5133,6010,5918,3324,3026
5,12.1 Personal care,4092,4274,4531,4697,4874,5072,5169,5017,5210,...,6163,6236,5916,6283,5945,6522,6733,6155,5970,5761
6,12.5 Insurance,5875,6206,6528,7519,8344,8809,8399,8292,8207,...,15398,14502,14820,18237,15276,14183,15225,15210,15887,19317


Renaming consumption and year column names:

In [280]:
cop.rename(columns = {'Unnamed: 2':'Variables'}, inplace=True)

col_dict = {}
col_dict = {str(i) : f'year{i}' for i in range(1994,2021+1)}
cop.rename(columns = col_dict, inplace=True)

cop.head(5)

,Variables,year1994,year1995,year1996,year1997,year1998,year1999,year2000,year2001,year2002,...,year2012,year2013,year2014,year2015,year2016,year2017,year2018,year2019,year2020,year2021
0,01.1 Food,22325,23280,23488,24021,24904,24905,25605,25803,25972,...,33133,32802,33381,33393,33268,33526,35068,34380,35485,34864
1,"04.5 Electricity, gas and other fuels",11551,11786,13261,14056,14669,14579,15426,17058,17774,...,23866,24282,24322,22306,20812,23919,23940,23580,19925,25635
2,05.2 Household textiles,920,916,852,870,851,907,915,1071,1272,...,1278,1215,1275,1496,1307,994,1064,1116,1408,1723
3,07.1 Purchase of vehicles,12170,12029,12453,13525,13535,14268,10072,9742,10985,...,14669,15307,13928,17255,15671,15465,17742,17754,21668,20965
4,07.3 Transport services,3663,3701,3445,3720,3425,3669,3869,3984,4154,...,6188,6096,5820,4841,4533,5133,6010,5918,3324,3026


Dropping unimportant variables:

In [281]:
# Build up a logical index I
I = cop.Variables.str.contains('Household textiles')
cop.loc[I, :]
cop = cop.loc[I == False] # keeping everything else
cop.head(10)

,Variables,year1994,year1995,year1996,year1997,year1998,year1999,year2000,year2001,year2002,...,year2012,year2013,year2014,year2015,year2016,year2017,year2018,year2019,year2020,year2021
0,01.1 Food,22325,23280,23488,24021,24904,24905,25605,25803,25972,...,33133,32802,33381,33393,33268,33526,35068,34380,35485,34864
1,"04.5 Electricity, gas and other fuels",11551,11786,13261,14056,14669,14579,15426,17058,17774,...,23866,24282,24322,22306,20812,23919,23940,23580,19925,25635
3,07.1 Purchase of vehicles,12170,12029,12453,13525,13535,14268,10072,9742,10985,...,14669,15307,13928,17255,15671,15465,17742,17754,21668,20965
4,07.3 Transport services,3663,3701,3445,3720,3425,3669,3869,3984,4154,...,6188,6096,5820,4841,4533,5133,6010,5918,3324,3026
5,12.1 Personal care,4092,4274,4531,4697,4874,5072,5169,5017,5210,...,6163,6236,5916,6283,5945,6522,6733,6155,5970,5761
6,12.5 Insurance,5875,6206,6528,7519,8344,8809,8399,8292,8207,...,15398,14502,14820,18237,15276,14183,15225,15210,15887,19317


Resetting index:

In [282]:
cop.reset_index(inplace = True, drop = True) # Drop old index too
cop.iloc[0:7,:]

,Variables,year1994,year1995,year1996,year1997,year1998,year1999,year2000,year2001,year2002,...,year2012,year2013,year2014,year2015,year2016,year2017,year2018,year2019,year2020,year2021
0,01.1 Food,22325,23280,23488,24021,24904,24905,25605,25803,25972,...,33133,32802,33381,33393,33268,33526,35068,34380,35485,34864
1,"04.5 Electricity, gas and other fuels",11551,11786,13261,14056,14669,14579,15426,17058,17774,...,23866,24282,24322,22306,20812,23919,23940,23580,19925,25635
2,07.1 Purchase of vehicles,12170,12029,12453,13525,13535,14268,10072,9742,10985,...,14669,15307,13928,17255,15671,15465,17742,17754,21668,20965
3,07.3 Transport services,3663,3701,3445,3720,3425,3669,3869,3984,4154,...,6188,6096,5820,4841,4533,5133,6010,5918,3324,3026
4,12.1 Personal care,4092,4274,4531,4697,4874,5072,5169,5017,5210,...,6163,6236,5916,6283,5945,6522,6733,6155,5970,5761
5,12.5 Insurance,5875,6206,6528,7519,8344,8809,8399,8292,8207,...,15398,14502,14820,18237,15276,14183,15225,15210,15887,19317


Removing numbers from consumption categories:

In [283]:
import string 
for value in cop['Variables'].values:
    cop.loc[cop['Variables'].values == value,['Variables']] = value.strip('0123456789.')

cop.loc[0,['Variables']] = 'Total consumption'

cop.head(10)

,Variables,year1994,year1995,year1996,year1997,year1998,year1999,year2000,year2001,year2002,...,year2012,year2013,year2014,year2015,year2016,year2017,year2018,year2019,year2020,year2021
0,Total consumption,22325,23280,23488,24021,24904,24905,25605,25803,25972,...,33133,32802,33381,33393,33268,33526,35068,34380,35485,34864
1,"Electricity, gas and other fuels",11551,11786,13261,14056,14669,14579,15426,17058,17774,...,23866,24282,24322,22306,20812,23919,23940,23580,19925,25635
2,Purchase of vehicles,12170,12029,12453,13525,13535,14268,10072,9742,10985,...,14669,15307,13928,17255,15671,15465,17742,17754,21668,20965
3,Transport services,3663,3701,3445,3720,3425,3669,3869,3984,4154,...,6188,6096,5820,4841,4533,5133,6010,5918,3324,3026
4,Personal care,4092,4274,4531,4697,4874,5072,5169,5017,5210,...,6163,6236,5916,6283,5945,6522,6733,6155,5970,5761
5,Insurance,5875,6206,6528,7519,8344,8809,8399,8292,8207,...,15398,14502,14820,18237,15276,14183,15225,15210,15887,19317


Set Index:

In [284]:
cop = cop.set_index('Variables')
cop.head()

,year1994,year1995,year1996,year1997,year1998,year1999,year2000,year2001,year2002,year2003,...,year2012,year2013,year2014,year2015,year2016,year2017,year2018,year2019,year2020,year2021
Variables,,,,,,,,,,,,,,,,,,,,,
Total consumption,22325,23280,23488,24021,24904,24905,25605,25803,25972,26428,...,33133,32802,33381,33393,33268,33526,35068,34380,35485,34864
"Electricity, gas and other fuels",11551,11786,13261,14056,14669,14579,15426,17058,17774,17939,...,23866,24282,24322,22306,20812,23919,23940,23580,19925,25635
Purchase of vehicles,12170,12029,12453,13525,13535,14268,10072,9742,10985,10423,...,14669,15307,13928,17255,15671,15465,17742,17754,21668,20965
Transport services,3663,3701,3445,3720,3425,3669,3869,3984,4154,4229,...,6188,6096,5820,4841,4533,5133,6010,5918,3324,3026
Personal care,4092,4274,4531,4697,4874,5072,5169,5017,5210,5314,...,6163,6236,5916,6283,5945,6522,6733,6155,5970,5761


Concatenate gdp and cop datasets:


In [285]:
#Check if they have the same variables 
different_years = [y for y in cop.columns.unique() if y not in gdp.columns.unique()] 
print(f'Columns (years) found in cop data but not in gdp: {different_years}')

#Concatenate them
all = pd.concat([cop,gdp])
all.head(50)

Columns (years) found in cop data but not in gdp: []


,year1994,year1995,year1996,year1997,year1998,year1999,year2000,year2001,year2002,year2003,...,year2012,year2013,year2014,year2015,year2016,year2017,year2018,year2019,year2020,year2021
Total consumption,22325,23280,23488,24021,24904,24905,25605,25803,25972,26428,...,33133,32802,33381,33393,33268,33526,35068,34380,35485,34864
"Electricity, gas and other fuels",11551,11786,13261,14056,14669,14579,15426,17058,17774,17939,...,23866,24282,24322,22306,20812,23919,23940,23580,19925,25635
Purchase of vehicles,12170,12029,12453,13525,13535,14268,10072,9742,10985,10423,...,14669,15307,13928,17255,15671,15465,17742,17754,21668,20965
Transport services,3663,3701,3445,3720,3425,3669,3869,3984,4154,4229,...,6188,6096,5820,4841,4533,5133,6010,5918,3324,3026
Personal care,4092,4274,4531,4697,4874,5072,5169,5017,5210,5314,...,6163,6236,5916,6283,5945,6522,6733,6155,5970,5761
Insurance,5875,6206,6528,7519,8344,8809,8399,8292,8207,10671,...,15398,14502,14820,18237,15276,14183,15225,15210,15887,19317
GDP,191,198,207,217,224,233,249,256,262,267,...,339,344,351,358,368,380,389,397,399,428


In [286]:
all.index.names = ['Variables']
all.head(10)

,year1994,year1995,year1996,year1997,year1998,year1999,year2000,year2001,year2002,year2003,...,year2012,year2013,year2014,year2015,year2016,year2017,year2018,year2019,year2020,year2021
Variables,,,,,,,,,,,,,,,,,,,,,
Total consumption,22325,23280,23488,24021,24904,24905,25605,25803,25972,26428,...,33133,32802,33381,33393,33268,33526,35068,34380,35485,34864
"Electricity, gas and other fuels",11551,11786,13261,14056,14669,14579,15426,17058,17774,17939,...,23866,24282,24322,22306,20812,23919,23940,23580,19925,25635
Purchase of vehicles,12170,12029,12453,13525,13535,14268,10072,9742,10985,10423,...,14669,15307,13928,17255,15671,15465,17742,17754,21668,20965
Transport services,3663,3701,3445,3720,3425,3669,3869,3984,4154,4229,...,6188,6096,5820,4841,4533,5133,6010,5918,3324,3026
Personal care,4092,4274,4531,4697,4874,5072,5169,5017,5210,5314,...,6163,6236,5916,6283,5945,6522,6733,6155,5970,5761
Insurance,5875,6206,6528,7519,8344,8809,8399,8292,8207,10671,...,15398,14502,14820,18237,15276,14183,15225,15210,15887,19317
GDP,191,198,207,217,224,233,249,256,262,267,...,339,344,351,358,368,380,389,397,399,428


Homogeneity in units:
Consumption is in DKK while GDP (per capita) is in 1000 DKK. It will be homogenized towards the unitary value.

In [287]:
scalar = 1000
all[all.index == "GDP"] = all[all.index == "GDP"] * scalar
all

,year1994,year1995,year1996,year1997,year1998,year1999,year2000,year2001,year2002,year2003,...,year2012,year2013,year2014,year2015,year2016,year2017,year2018,year2019,year2020,year2021
Variables,,,,,,,,,,,,,,,,,,,,,
Total consumption,22325,23280,23488,24021,24904,24905,25605,25803,25972,26428,...,33133,32802,33381,33393,33268,33526,35068,34380,35485,34864
"Electricity, gas and other fuels",11551,11786,13261,14056,14669,14579,15426,17058,17774,17939,...,23866,24282,24322,22306,20812,23919,23940,23580,19925,25635
Purchase of vehicles,12170,12029,12453,13525,13535,14268,10072,9742,10985,10423,...,14669,15307,13928,17255,15671,15465,17742,17754,21668,20965
Transport services,3663,3701,3445,3720,3425,3669,3869,3984,4154,4229,...,6188,6096,5820,4841,4533,5133,6010,5918,3324,3026
Personal care,4092,4274,4531,4697,4874,5072,5169,5017,5210,5314,...,6163,6236,5916,6283,5945,6522,6733,6155,5970,5761
Insurance,5875,6206,6528,7519,8344,8809,8399,8292,8207,10671,...,15398,14502,14820,18237,15276,14183,15225,15210,15887,19317
GDP,191000,198000,207000,217000,224000,233000,249000,256000,262000,267000,...,339000,344000,351000,358000,368000,380000,389000,397000,399000,428000


Check consumption of each variable over GDP:


In [288]:
for val in all.index:
    all.loc[val + "/GDP"] = all.loc[val] / all.loc["GDP"]
all
#All values have turned to floats, which is slightly unpleasant. Maybe it could be fixed 
#by changing all values to "object" type and setting the upper half to int and the lower to float,
#but that would defeat the purpose of a panda df

,year1994,year1995,year1996,year1997,year1998,year1999,year2000,year2001,year2002,year2003,...,year2012,year2013,year2014,year2015,year2016,year2017,year2018,year2019,year2020,year2021
Variables,,,,,,,,,,,,,,,,,,,,,
Total consumption,22325.000000,23280.000000,23488.000000,24021.000000,24904.000000,24905.000000,25605.000000,25803.000000,25972.000000,26428.000000,...,33133.000000,32802.000000,33381.000000,33393.000000,33268.000000,33526.000000,35068.000000,34380.000000,35485.000000,34864.000000
"Electricity, gas and other fuels",11551.000000,11786.000000,13261.000000,14056.000000,14669.000000,14579.000000,15426.000000,17058.000000,17774.000000,17939.000000,...,23866.000000,24282.000000,24322.000000,22306.000000,20812.000000,23919.000000,23940.000000,23580.000000,19925.000000,25635.000000
Purchase of vehicles,12170.000000,12029.000000,12453.000000,13525.000000,13535.000000,14268.000000,10072.000000,9742.000000,10985.000000,10423.000000,...,14669.000000,15307.000000,13928.000000,17255.000000,15671.000000,15465.000000,17742.000000,17754.000000,21668.000000,20965.000000
Transport services,3663.000000,3701.000000,3445.000000,3720.000000,3425.000000,3669.000000,3869.000000,3984.000000,4154.000000,4229.000000,...,6188.000000,6096.000000,5820.000000,4841.000000,4533.000000,5133.000000,6010.000000,5918.000000,3324.000000,3026.000000
Personal care,4092.000000,4274.000000,4531.000000,4697.000000,4874.000000,5072.000000,5169.000000,5017.000000,5210.000000,5314.000000,...,6163.000000,6236.000000,5916.000000,6283.000000,5945.000000,6522.000000,6733.000000,6155.000000,5970.000000,5761.000000
Insurance,5875.000000,6206.000000,6528.000000,7519.000000,8344.000000,8809.000000,8399.000000,8292.000000,8207.000000,10671.000000,...,15398.000000,14502.000000,14820.000000,18237.000000,15276.000000,14183.000000,15225.000000,15210.000000,15887.000000,19317.000000
GDP,191000.000000,198000.000000,207000.000000,217000.000000,224000.000000,233000.000000,249000.000000,256000.000000,262000.000000,267000.000000,...,339000.000000,344000.000000,351000.000000,358000.000000,368000.000000,380000.000000,389000.000000,397000.000000,399000.000000,428000.000000
Total consumption/GDP,0.116885,0.117576,0.113469,0.110696,0.111179,0.106888,0.102831,0.100793,0.099130,0.098981,...,0.097737,0.095355,0.095103,0.093277,0.090402,0.088226,0.090149,0.086599,0.088935,0.081458
"Electricity, gas and other fuels/GDP",0.060476,0.059525,0.064063,0.064774,0.065487,0.062571,0.061952,0.066633,0.067840,0.067187,...,0.070401,0.070587,0.069293,0.062307,0.056554,0.062945,0.061542,0.059395,0.049937,0.059895


Next things to do: 

- **Solving problem about real prices base year:** <br> 
    Base year for gdp = 2010 <br>
    Base year for cop = 2015 <br>
   We solve the problem by using current prices. - <font color=green> Done </font> <br>
   **Problem with current prices:** GDP is in million of DKK while consumption is just in DKK - <font color=green> Done </font> <br>
   Solved with:
   - changed parameters: gdp is now in current per capita prices (so no million DKK problem);<br>
   - imported new excel file FU07_cp: cop is now at current prices
- Drop remaining NaN rows in cop dataset - <font color=green>Done</font>
- New index for cop dataset - <font color=green>Done</font>
- Trasform gdp dataset: 
    - years as column names - <font color=green>Done</font>
    - only one row called gdp: drop other - <font color=green>Done</font>
- Concat. datasets - <font color=green>Done</font> 
- Create new data as a result of running an operator on other data in the dataset:
    (e.g. summing two rows) - <font color=green>Done</font> (But the comment should be considered)
- Running a function on the dataset - <font color=red>To do</font>
- Modify GDP values of Consumption value so that is there homogeneity of unit measure (now: Consumption is in dkk and and GDP in 1000 Dkk) - <font color=green>Done</font> (albeit the other way around)